In [1]:
from skimage import io
from skimage.viewer import ImageViewer
import pandas as pd
from sklearn import linear_model
import math
import numpy as np
import skimage
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier
from sklearn import cross_validation
from sklearn import svm
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

In [2]:
img_data = pd.read_csv('Final_data.csv',header=None)
img_data = img_data[:]
data_size = img_data.shape[0]
img_data.head()

,0,1,2,3,4,5,6
0,mdb001,G,CIRC,B,535,425,197
1,mdb002,G,CIRC,B,522,280,69
2,mdb003,D,NORM,N,512,512,100
3,mdb004,D,NORM,N,512,512,100
4,mdb005,F,CIRC,B,477,133,30


In [3]:
import copy
def get_glcm_features(angle_code, dist,Z): ## angle_code 0-->1, 45-->2, 90-->3, 135-->4, 180-->5
    X=copy.deepcopy(Z)
    fa1,fb1,fc1,fd1 = np.zeros(data_size),np.zeros(data_size),np.zeros(data_size),np.zeros(data_size)
    img_name = list(img_data[0])
    for i in range(len(img_data)) :
        name = str(img_name[i])+".pgm"
        I = io.imread('Final_Dataset/'+name)
        if angle_code==1:
            glcm = skimage.feature.greycomatrix(I, [dist], [0], normed=True)
        elif angle_code==2:
            glcm = skimage.feature.greycomatrix(I, [dist], [-dist], normed=True)
        elif angle_code==3:
            glcm = skimage.feature.greycomatrix(I, [0], [-dist], normed=True)
        elif angle_code==4:
            glcm = skimage.feature.greycomatrix(I, [-dist], [-dist], normed=True)
        elif angle_code==5:
            glcm = skimage.feature.greycomatrix(I, [-dist], [0], normed=True)

        fa1[i] = skimage.feature.greycoprops(glcm, 'contrast')[0][0]
        fb1[i] = skimage.feature.greycoprops(glcm, 'energy')[0][0]
        fc1[i] = skimage.feature.greycoprops(glcm, 'homogeneity')[0][0]
        fd1[i] = skimage.feature.greycoprops(glcm, 'correlation')[0][0]
    X['contrast_ang'+str(angle_code)+'_d'+str(dist)],X['energy_ang'+str(angle_code)+'_d'+str(dist)],X['homogeneity_ang'+str(angle_code)+'_d'+str(dist)],X['correlation_ang'+str(angle_code)+'_d'+str(dist)]=fa1,fb1,fc1,fd1
    return X

In [4]:
def indices_list(l) :
    ans=[l[0],l[1],l[2],l[3]]
    indices=[0,1,2,3]
    for i in range(4,len(l)) :
        if l[i]>ans[0]:
            ans[3]=ans[2]
            ans[2]=ans[1]
            ans[1]=ans[0]
            ans[0]=l[i]
            indices[3]=indices[2]
            indices[2]=indices[1]
            indices[1]=indices[0]
            indices[0]=i
        elif l[i]>ans[1]:
            ans[3]=ans[2]
            ans[2]=ans[1]
            ans[1]=l[i]
            indices[3]=indices[2]
            indices[2]=indices[1]
            indices[1]=i
        elif l[i]>ans[2]:
            ans[3]=ans[2]
            ans[2]=l[i]
            indices[3]=indices[2]
            indices[2]=i
        elif l[i]>ans[3]:
            ans[3]=l[i]
            indices[3]=i
    return indices

In [5]:
def get_accuracy(X,y,trees) :
    model = linear_model.LogisticRegression()
    return cross_validation.cross_val_score(model,X,y,cv=5).mean()

In [6]:
def get_accuracy_rf(X,y,trees) :
    model = GradientBoostingClassifier(n_estimators=300)
    return cross_validation.cross_val_score(model,X,y,cv=5).mean()

In [7]:
Y = img_data[3].apply(lambda x : 0 if x=='N' else 1)
X = pd.DataFrame()
L=[]
for i in range(1,4) :
    l=[]
    for j in range(1,30) : 
        X = get_glcm_features(i,j,pd.DataFrame())
        l.append(get_accuracy(X,Y,150))
    L.append(l)

In [8]:
for i in range(len(L)):
    plt.plot(range(1,30),L[i])
plt.show()

In [9]:
X = pd.DataFrame()
for i in range(1,4):
    indices = indices_list(L[i-1])
    for j in indices:
        X = get_glcm_features(i,j+1,X)
print get_accuracy(X,Y,100)

0.800932400932


In [10]:
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
model6 = AdaBoostClassifier(n_estimators=500)
model6.fit(X,Y)
x = 100*model6.feature_importances_
print x

[ 1.4  0.   1.2  0.   0.6  0.   0.2  1.2  2.2  3.8  2.2  1.6  0.6  0.8  1.8
  3.6  0.8  5.   4.2  5.4  1.2  7.6  4.2  3.6  7.2  3.2  6.4  8.4  1.8  1.6
  4.4  5.8  0.   1.6  0.   0.   0.   1.8  0.   0.   0.   2.6  0.   0.   0.
  2.   0.   0. ]


In [11]:
Q=X.iloc[:,x>=4]
Q.head()#graph analysis

,energy_ang2_d21,homogeneity_ang2_d21,correlation_ang2_d21,energy_ang2_d24,homogeneity_ang2_d24,contrast_ang2_d8,homogeneity_ang2_d8,correlation_ang2_d8,homogeneity_ang2_d18,correlation_ang2_d18
0,0.019972,0.102839,0.944508,0.019702,0.091644,113.922638,0.183015,0.987131,0.109730,0.954619
1,0.033220,0.109895,0.918456,0.033546,0.109834,143.535402,0.173735,0.973997,0.112020,0.930794
2,0.024496,0.094463,0.961822,0.024758,0.104899,69.258912,0.194261,0.990931,0.096271,0.964419
3,0.030719,0.143517,0.963155,0.030019,0.146202,40.232746,0.209240,0.989735,0.150423,0.960981
4,0.032878,0.024326,0.148216,0.033080,0.021774,275.510377,0.064017,0.838555,0.027860,0.312995


In [47]:
from sklearn.decomposition import PCA
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X,Y, test_size=0.2, random_state=3)
Qtrain=X_train.iloc[:,x>=3]
Qtest=X_test.iloc[:,x>=3]
pca = PCA(n_components=15).fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
model = GradientBoostingClassifier(n_estimators=500)
print get_accuracy_cvset(model,X_train_pca,X_test_pca,y_train,y_test)
print get_accuracy_cvset(model,X_train,X_test,y_train,y_test)
print get_accuracy_cvset(model,Qtrain,Qtest,y_train,y_test)

25
0.863636363636
22
0.878787878788
23
0.924242424242


In [42]:
print len(y_test[y_test==0]) , len(y_test[y_test==1])

42 24


In [48]:
print get_accuracy(Q,Y,150)
print get_accuracy(X,Y,150)

0.797948717949
0.800932400932


In [165]:
l=[]
for i in range(1,201,10) : 
    l.append(get_accuracy(Q,Y,i))
plt.plot(range(1,201,10),l)
plt.show()

In [5]:
features = ['contrast1','energy1','homogeneity1','correlation1','contrast2','energy2','homogeneity2','correlation2']

In [21]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X,Y, test_size=0.2, random_state=3)

In [13]:
def get_accuracy_cvset(model,x_train,x_test,y_train,y_test) :
    model.fit(x_train,y_train)
    ypred = model.predict(x_test)
    y_test=np.array(y_test)
    mis=0
    for i in range(len(ypred)) : mis += abs(ypred[i]-y_test[i])
    print len(ypred[ypred==1])
    return (len(ypred)-mis)/(len(ypred)+0.00)

In [220]:
model1 = linear_model.LogisticRegression()
scores = cross_validation.cross_val_score(model1,X,Y,cv=5)
print scores.mean()
print get_accuracy(model1,X_train,X_test,y_train,y_test)
#print get_accuracy(model1,X_train_pca,X_test_pca,y_train,y_test)

0.701148105626
25
0.80303030303


In [222]:
model2 = RandomForestClassifier(n_estimators=10,min_samples_leaf=5,min_samples_split=10)
scores = cross_validation.cross_val_score(model2,X,Y,cv=20)
print scores.mean()
print get_accuracy(model2,X_train,X_test,y_train,y_test)
#print get_accuracy(model2,X_train_pca,X_test_pca,y_train,y_test)

0.710902777778
23
0.924242424242


In [94]:
model3 = svm.SVC(kernel='linear', C=1)
scores = cross_validation.cross_val_score(model3,X,Y,cv=5)
print scores.mean()
print get_accuracy(model3,X_train,X_test,y_train,y_test)

0.797948717949
25
0.833333333333


In [154]:
def predict_by_ensemble(mode1,model2,model3,X_test):
    x1=model1.predict(X_test)
    x2=model2.predict(X_test)
    x3=model3.predict(X_test)
    y_pred=x1+x2+x3
    for i in range(len(y_pred)) :
        if y_pred[i]>1 : y_pred[i]=1
        else : y_pred[i]=0
    return y_pred

In [198]:
ypred = predict_by_ensemble(model2,model4,model6,X_test)
y_test=np.array(y_test)
mis=0
for i in range(len(ypred)) :
    mis += abs(ypred[i]-y_test[i])
print len(ypred[ypred==1])
print (len(ypred)-mis)/(len(ypred)+0.00)

20
0.909090909091


In [229]:
model4 = GradientBoostingClassifier(n_estimators=100,min_samples_leaf=5)
scores = cross_validation.cross_val_score(model4,X,Y,cv=20)
print scores.mean()
print get_accuracy(model4,X_train,X_test,y_train,y_test)
#print get_accuracy(model4,X_train_pca,X_test_pca,y_train,y_test)

0.456904761905
22
0.909090909091


In [119]:
import xgboost as xgb
gbm = xgb.XGBClassifier(n_estimators=10)
scores = cross_validation.cross_val_score(gbm,X,Y)
print scores.mean()
print get_accuracy(gbm,X_train,X_test,y_train,y_test)

0.859327217125
23
0.924242424242


In [ ]:
from sklearn.decomposition import PCA
from sklearn.svm import SVC
pca = PCA(n_components=6).fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
clf = SVC(kernel='linear',C=1)
print get_accuracy(clf,X_train_pca,X_test_pca,y_train,y_test)

In [103]:
from sklearn.tree import DecisionTreeClassifier
model5 = DecisionTreeClassifier()
scores = cross_validation.cross_val_score(model5,X,Y,cv=20)
print scores.mean()
print get_accuracy(model5,X_train,X_test,y_train,y_test)

0.838419117647
27
0.833333333333


In [12]:
model2 = RandomForestClassifier(n_estimators=11)
model2.fit(X_train,y_train)
pred=model2.predict(X_test)
print 'Accuracy:', accuracy_score(y_test,pred)
conf_mtr = confusion_matrix(y_test, pred)
print conf_mtr
plt.matshow(conf_mtr)
plt.title('Confusion matrix')
plt.colorbar()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

Accuracy: 0.737373737374
[[72  6]
 [20  1]]


In [9]:
len(Y[Y==0])

207

In [55]:
len(y_train)

228

In [195]:
foo = Y==1

In [107]:
zeros , ones = X_train[y_train==0] , X_train[y_train==1]
X1 = X_train
X1 = X_train.append(ones)
X1 = X1.append(ones)
Y1 = np.concatenate([y_train,np.ones(106)])
gbm = GradientBoostingClassifier(n_estimators=1000,min_samples_leaf=5,min_samples_split=10)
scores = cross_validation.cross_val_score(gbm,X1[features],Y1,cv=2)
print scores.mean()
print get_accuracy(gbm,X1[features],X_test[features],Y1,y_test)

0.890977666904
0.772727272727


In [108]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
scores = cross_validation.cross_val_score(dtc,X1[features],Y1,cv=2)
scores.mean()
In [6]:
￼
Y = img_data[3].apply(lambda x : 0 if x=='N' else 1)

0.88018234735091472

In [231]:
len(Y[Y==0])

207

In [232]:
len(Y[Y==1])

120